# Pre-processing
## Base_DF

In [7]:
import psycopg2
from sqlalchemy import create_engine
from pathlib import Path
import json
DB_CONFIG_PATH = Path(Path.cwd().parent.parent.parent.parent/"settings"/"db_config.json")
with open(str(DB_CONFIG_PATH), "r") as read_file: DB_CONFIG = json.load(read_file)

import pandas as pd
# Replace 'username', 'password', 'host', 'port', and 'database' with your actual database credentials
engine = create_engine('postgresql' + '://{user}:{password}@{host}:{port}/{database}'.format(**DB_CONFIG))


ModuleNotFoundError: No module named 'psycopg2'

In [2]:
df_report_links_raw = pd.read_sql_query("""
SELECT *
FROM tickers_table 
INNER JOIN 
        report_liks ON report_liks.cik = tickers_table.cik

ORDER BY tickers_table.cik"""
    ,engine) 



In [3]:
# pd.to_datetime(df_report_links_raw['date_publication','date_added'],format='%Y-%m-%d')
df_report_links_raw[['date_publication','date_added']] = df_report_links_raw[['date_publication','date_added']].apply(pd.to_datetime)


In [7]:
# df_report_links_raw

In [4]:
df_descriptive = pd.read_sql_table(
    table_name='descriptive_info',con=engine)

In [5]:
df_descriptive.fullTimeEmployees = df_descriptive.fullTimeEmployees.apply(pd.to_numeric)

# Main Grouped

In [6]:
grouped_reports = (df_report_links_raw
    .groupby('ticker',as_index=False).agg(
        unique_url = ('url_report',pd.Series.nunique))
    .sort_values(by="unique_url",ascending=False)    
    .rename(columns={"ticker":'symbol'})    )
grouped_reports

,symbol,unique_url
7859,VAL-WT,65
7858,VAL,65
1318,BTCS,64
3210,GE,62
5090,MSI,60
...,...,...
3791,HUDA,1
7919,VHLD,1
7099,SOYB,1
6701,SATL,1


In [7]:
grouped_desc_reports = (
    df_descriptive.merge(grouped_reports,on='symbol')
)
grouped_desc_reports.head()

,id,symbol,longName,sector,industry,country,longBusinessSummary,fullTimeEmployees,website,logo_url,exchange,unique_url
0,1,A,"Agilent Technologies, Inc.",Healthcare,Diagnostics & Research,United States,"Agilent Technologies, Inc. provides applicatio...",17200.0,https://www.agilent.com,https://logo.clearbit.com/agilent.com,NYQ,57
1,2,AA,Alcoa Corporation,Basic Materials,Aluminum,United States,"Alcoa Corporation, together with its subsidiar...",12200.0,https://www.alcoa.com,https://logo.clearbit.com/alcoa.com,NYQ,26
2,3,AAC,Ares Acquisition Corporation,Financial Services,Shell Companies,United States,Ares Acquisition Corporation does not have sig...,NaN,https://www.aresacquisitioncorporation.com,https://logo.clearbit.com/aresacquisitioncorpo...,NYQ,10
3,4,AACG,ATA Creativity Global,Consumer Defensive,Education & Training Services,China,"ATA Creativity Global, together with its subsi...",NaN,https://www.atai.net.cn,https://logo.clearbit.com/atai.net.cn,NGM,10
4,5,AACI,Armada Acquisition Corp. I,Financial Services,Shell Companies,United States,Armada Acquisition Corp. I focuses on effectin...,NaN,None,,NGM,8


In [8]:
df_companies = grouped_desc_reports[['symbol',"sector",'industry','country',"longName","longBusinessSummary",'fullTimeEmployees','unique_url']]

In [9]:
df_companies.sector.value_counts()
df_companies

,symbol,sector,industry,country,longName,longBusinessSummary,fullTimeEmployees,unique_url
0,A,Healthcare,Diagnostics & Research,United States,"Agilent Technologies, Inc.","Agilent Technologies, Inc. provides applicatio...",17200.0,57
1,AA,Basic Materials,Aluminum,United States,Alcoa Corporation,"Alcoa Corporation, together with its subsidiar...",12200.0,26
2,AAC,Financial Services,Shell Companies,United States,Ares Acquisition Corporation,Ares Acquisition Corporation does not have sig...,NaN,10
3,AACG,Consumer Defensive,Education & Training Services,China,ATA Creativity Global,"ATA Creativity Global, together with its subsi...",NaN,10
4,AACI,Financial Services,Shell Companies,United States,Armada Acquisition Corp. I,Armada Acquisition Corp. I focuses on effectin...,NaN,8
...,...,...,...,...,...,...,...,...
5540,ZVIA,Consumer Defensive,Beverages—Non-Alcoholic,United States,Zevia PBC,"Zevia PBC, a beverage company, develops, marke...",119.0,7
5541,ZWS,Utilities,Utilities—Regulated Water,United States,Zurn Water Solutions Corporation,"Zurn Water Solutions Corporation designs, proc...",1300.0,44
5542,ZYME,Healthcare,Biotechnology,Canada,Zymeworks Inc.,"Zymeworks Inc., a clinical-stage biopharmaceut...",455.0,2
5543,ZYNE,Healthcare,Drug Manufacturers—Specialty & Generic,United States,"Zynerba Pharmaceuticals, Inc.","Zynerba Pharmaceuticals, Inc. operates as a cl...",28.0,31


In [10]:
worked_df = (
    df_companies
    .query("unique_url >40")
    .groupby(["industry",'sector']).head(5)
)
# worked_df


# Processing


In [11]:
"""
    Get 5 first sector
    REQUIREMENT: setor
"""

selected_first_5 = (
    worked_df.query('sector == "Healthcare"')
    .sort_values(by=['industry'],ascending=False))

In [12]:
pd.set_option('display.max_colwidth', 50)
 
test_group_df = (
    
    pd.merge(
        df_report_links_raw
            [["ticker",'date_publication','quartal','url_report']],
        selected_first_5.rename(columns={"symbol":'ticker'})
            [['sector','industry','ticker']],
    on='ticker'
    )
    .drop_duplicates().sort_values(by=['ticker',"date_publication"])
    # .style.set_properties(**{'text-align': 'center', 'text_wrap': True}) 
)
test_group_tickers= test_group_df.ticker.unique()
test_group_tickers

array(['A', 'ABC', 'ABEO', 'ABIO', 'ABT', 'ABVC', 'ACAD', 'ACER', 'ACHC',
       'ACOR', 'ACRX', 'ADMP', 'ADUS', 'AEMD', 'AHPI', 'ALGN', 'ALIM',
       'ALR', 'AMED', 'AMEH', 'ANGO', 'ANIK', 'ANIP', 'APDN', 'ATR',
       'ATRC', 'ATRI', 'AWH', 'AXDX', 'AZTA', 'BIIB', 'BIMI', 'BMY',
       'CAH', 'CEMI', 'CNC', 'CPSI', 'CVS', 'GILD', 'HSIC', 'HSTM',
       'HZNP', 'JNJ', 'LFMD', 'LFMDP', 'MCK', 'MDRX', 'MDVL', 'NRC',
       'NXGN', 'OMI', 'PETS', 'UNH'], dtype=object)

# get

In [13]:
# [0]
def get_list(_index):
    df= test_group_df.query(f'ticker == @test_group_tickers[{_index}]').head(6)
        
    ticker = str(df.ticker.iloc[0])
    utls_list = df.url_report.to_list()
    return ticker,utls_list

In [14]:
docs = get_list(2)
[
    x+'/Financial_Report.xls'
    for x in 
    docs[1]
    ]

['https://www.sec.gov/Archives/edgar/data/0000318306/000031830611000014/Financial_Report.xls',
 'https://www.sec.gov/Archives/edgar/data/0000318306/000031830611000025/Financial_Report.xls',
 'https://www.sec.gov/Archives/edgar/data/0000318306/000031830612000020/Financial_Report.xls',
 'https://www.sec.gov/Archives/edgar/data/0000318306/000031830612000036/Financial_Report.xls',
 'https://www.sec.gov/Archives/edgar/data/0000318306/000031830612000048/Financial_Report.xls',
 'https://www.sec.gov/Archives/edgar/data/0000318306/000031830612000057/Financial_Report.xls']